In [213]:
import pandas as pd
import numpy as np
import datetime
import math
import sklearn.cross_validation
from scipy import stats

In [214]:
train = pd.read_csv('data_original/train.csv')
test = pd.read_csv('data_original/test.csv')

print(len(train.Point.unique()))
print(len(test.Point.unique()))
a = set(train.Point.unique()) - (set(train.Point.unique()) & set(test.Point.unique()))
for i in a:
    print(i)
    train = train[train.Point != i]

    
test['Count'] = None
frame = [train, test]
common = pd.concat(frame)
common.Date = common.Date.apply(lambda x: pd.to_datetime(x, format='%d.%m.%Y'))
common = common.sort_values(by=['Date'])

114
114


In [215]:
common.to_csv('data_original/common.csv', index=None)
train.to_csv('data_original/train.csv', index=None)
test.to_csv('data_original/test.csv', index=None)

# date.csv

In [216]:
date = pd.read_csv('data_original/common.csv')[['Date']]
date.Date = date.Date.apply(lambda x: pd.to_datetime(x))

In [217]:
start_date = date.iloc[0]['Date']
date['Year'] = date.Date.apply(lambda x: x.year)
date['Month'] = date.Date.apply(lambda x: x.month)
date['MonthDay'] = date.Date.apply(lambda x: x.day)
date['WeekDay'] = date.Date.apply(lambda x: x.weekday())
date['Point'] = pd.Series(common.Point.as_matrix())

In [218]:
date['YearDay'] = date.Date.apply(lambda x: (x-pd.to_datetime(str(x.year)+'-01-01')).days)

In [219]:
date['ThroughMonth'] = date.Date.apply(lambda x: (x.year - start_date.year) * 12 + x.month - start_date.month)

In [220]:
date['ThroughWeek'] = date.Date.apply(lambda x: int( ( (x - start_date).days + start_date.weekday()) / 7))

In [221]:
date['ThroughDay'] = date.Date.apply(lambda x: (x - start_date).days)

In [222]:
date.to_csv('data_transform/date.csv', index=None)

In [223]:
date.head(10)

,Date,Year,Month,MonthDay,WeekDay,Point,YearDay,ThroughMonth,ThroughWeek,ThroughDay
0,2011-10-06,2011,10,6,3,Сыктывкар-1,278,0,0,0
1,2011-10-07,2011,10,7,4,Сыктывкар-1,279,0,0,1
2,2011-10-09,2011,10,9,6,Сыктывкар-1,281,0,0,3
3,2011-11-15,2011,11,15,1,Сыктывкар-1,318,1,6,40
4,2011-12-08,2011,12,8,3,Сыктывкар-1,341,2,9,63
5,2011-12-09,2011,12,9,4,Сыктывкар-1,342,2,9,64
6,2011-12-10,2011,12,10,5,Сыктывкар-1,343,2,9,65
7,2011-12-11,2011,12,11,6,Сыктывкар-1,344,2,9,66
8,2011-12-13,2011,12,13,1,Сыктывкар-1,346,2,10,68
9,2011-12-14,2011,12,14,2,Сыктывкар-1,347,2,10,69


# target.csv

In [224]:
sizeValidationSample = 0.16

In [225]:
target = pd.read_csv('data_original/common.csv')[['Count', 'Date']]
#target['Point'] = pd.Series(common.Point.as_matrix())
target['IsTest'] = target.Count
target['IsTrain'] = pd.Series(range(target.shape[0]))
target['IsValidation'] = pd.Series(range(target.shape[0]))

In [226]:
target['IsTest'] = target.IsTest.apply(lambda x: 1 if math.isnan(x) else 0)

In [227]:
withCountValue = target[target.IsTest == 0].index
indexTrain, indexValidation = sklearn.cross_validation.train_test_split(withCountValue, test_size=sizeValidationSample)

In [228]:
dates = target.Date[target.IsTest == 1]
test_date_count = dates.unique().shape[0]
print(test_date_count)
dates = target.Date[target.IsTest == 0].unique()
np.random.shuffle(dates)
validation_dates = dates[:test_date_count]
validation_dates
target['IsValidation'] = target.Date.apply(lambda x: 1 if x in validation_dates else 0)
target.head(20)

268


,Count,Date,IsTest,IsTrain,IsValidation
0,8.0,2011-10-06,0,0,1
1,88.0,2011-10-07,0,1,0
2,1.0,2011-10-09,0,2,0
3,1.0,2011-11-15,0,3,1
4,2.0,2011-12-08,0,4,0
5,43.0,2011-12-09,0,5,1
6,10.0,2011-12-10,0,6,0
7,1.0,2011-12-11,0,7,0
8,60.0,2011-12-13,0,8,0
9,NaN,2011-12-14,1,9,0


In [229]:
target['IsTrain'] = 1
indexes = target[np.logical_or(target.IsValidation == 1, target.IsTest == 1)].index
target.IsTrain.loc[indexes] = 0
#target['IsValidation'] = target.IsValidation.apply(lambda x: 1 if ((x in indexValidation) & (target.IsValidation[x] == 0)) else 0)

In [230]:
target.to_csv('data_transform/target.csv', index = None)

In [231]:
CountWithouthNaNValueValidation = target.Count.copy()

In [232]:
target.Count.loc[target.Count[target.IsValidation == 1].index] = np.nan


In [233]:
target.head(20)

,Count,Date,IsTest,IsTrain,IsValidation
0,NaN,2011-10-06,0,0,1
1,88.0,2011-10-07,0,1,0
2,1.0,2011-10-09,0,1,0
3,NaN,2011-11-15,0,0,1
4,2.0,2011-12-08,0,1,0
5,NaN,2011-12-09,0,0,1
6,10.0,2011-12-10,0,1,0
7,1.0,2011-12-11,0,1,0
8,60.0,2011-12-13,0,1,0
9,NaN,2011-12-14,1,0,0


# city.csv


In [234]:
cities = pd.read_csv('data_original/common.csv')[['Point']].Point.unique()
city = pd.DataFrame()
city['Point'] = pd.Series(cities)
city['CityName'] = city.Point.apply(lambda x: '-'.join(x.split('-')[0:-1]))
city_names = city.CityName.unique()
city['CityID'] = city.CityName.apply(lambda x: np.where(city_names == x)[0][0]) 
city['BranchNumber'] = city.Point.apply(lambda x: x.split('-')[-1:][0])
# Москва 1 -> Москва
city['CityName'] = city.CityName.apply(lambda x: ('Москва' if x == 'Москва 1' else x))
city.to_csv('data_transform/city.csv', index = None)

In [235]:
city.head(10)

,Point,CityName,CityID,BranchNumber
0,Сыктывкар-1,Сыктывкар,0,1
1,Сыктывкар-2,Сыктывкар,0,2
2,Ухта-1,Ухта,1,1
3,Смоленск-1,Смоленск,2,1
4,Сарапул-1,Сарапул,3,1
5,Самара-1,Самара,4,1
6,Челябинск-1,Челябинск,5,1
7,Белгород-1,Белгород,6,1
8,Смоленск-2,Смоленск,2,2
9,Самара-2,Самара,4,2


# internal_features.csv

In [236]:
src = pd.read_csv('data_transform/date.csv')
src['Point'] = pd.read_csv('data_original/common.csv')['Point']
src['Count'] = target.Count
dst = pd.DataFrame() 

In [237]:
src.head(15)

,Date,Year,Month,MonthDay,WeekDay,Point,YearDay,ThroughMonth,ThroughWeek,ThroughDay,Count
0,2011-10-06,2011,10,6,3,Сыктывкар-1,278,0,0,0,NaN
1,2011-10-07,2011,10,7,4,Сыктывкар-1,279,0,0,1,88.0
2,2011-10-09,2011,10,9,6,Сыктывкар-1,281,0,0,3,1.0
3,2011-11-15,2011,11,15,1,Сыктывкар-1,318,1,6,40,NaN
4,2011-12-08,2011,12,8,3,Сыктывкар-1,341,2,9,63,2.0
5,2011-12-09,2011,12,9,4,Сыктывкар-1,342,2,9,64,NaN
6,2011-12-10,2011,12,10,5,Сыктывкар-1,343,2,9,65,10.0
7,2011-12-11,2011,12,11,6,Сыктывкар-1,344,2,9,66,1.0
8,2011-12-13,2011,12,13,1,Сыктывкар-1,346,2,10,68,60.0
9,2011-12-14,2011,12,14,2,Сыктывкар-1,347,2,10,69,NaN


## Next-Prev N Existing Count

In [238]:
tmp = pd.DataFrame()
tmp[["Count", "IsTest", "IsTrain", "IsValidation"]]= target[["Count", "IsTest", "IsTrain", "IsValidation"]]
tmp["Point"] = date.Point
tmp["Date"] = pd.to_datetime(date.Date)

tmp.head(10)

resultExistingMean = pd.DataFrame()

In [239]:
def GetNextNExistingValue(n):
    result = np.array([np.nan] * tmp.shape[0])
    k = 1
    for i in range(tmp.shape[0]):
        currentDay = tmp.Date[i]
        targetCity = tmp.Point[i]
        tmpForCity = tmp[(tmp.Point == targetCity) &(tmp.IsTrain == 1)] #получение данных только по определенному 
        #городу и из Train части
        valueLessWhenN = (tmpForCity.Date.map(lambda x: (x  - currentDay).days) <= n) \
                        &(tmpForCity.Date.map(lambda x: (x  - currentDay).days) > 0)
        # получение городов, чья разница между первоночальной датой и вычисляемой меньше n и больше 0 (в днях)
        targetCount = tmpForCity.Count[valueLessWhenN] #получение Count переменных для этих городов
        targetCount = targetCount.dropna() #удаление отсутвующих значений для вычисления средних
        meanCount = np.mean(targetCount)
        result[i] = meanCount
        k += 1
        if ( (k % 500) == 1):
            print k / 500,
    return result

def GetPrevNExistingValue(n):
    result = np.array([np.nan] * tmp.shape[0])
    k = 1
    for i in range(tmp.shape[0]):
        currentDay = tmp.Date[i]
        targetCity = tmp.Point[i]
        tmpForCity = tmp[(tmp.Point == targetCity) &(tmp.IsTrain == 1)] #получение данных только по определенному 
        #городу и из Train части
        valueLessWhenN = (tmpForCity.Date.map(lambda x: (x  - currentDay).days) >= n) \
                        &(tmpForCity.Date.map(lambda x: (x  - currentDay).days) < 0)
        # получение городов, чья разница между первоночальной датой и вычисляемой меньше n и больше 0 (в днях)
        targetCount = tmpForCity.Count[valueLessWhenN] #получение Count переменных для этих городов
        targetCount = targetCount.dropna() #удаление отсутвующих значений для вычисления средних
        meanCount = np.mean(targetCount)
        result[i] = meanCount
        k += 1
        if ( (k % 500) == 1):
            print k / 500,
    return result

In [240]:
resultExistingMean["NextExisting1Value"] = pd.Series(GetNextNExistingValue(1))

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106


In [241]:
resultExistingMean["NextExisting2Value"] = pd.Series(GetNextNExistingValue(2))

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106


In [242]:
resultExistingMean["NextExisting3Value"] = pd.Series(GetNextNExistingValue(3))

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106


In [243]:
resultExistingMean["NextExisting7Value"] = pd.Series(GetNextNExistingValue(7))

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106


In [244]:
resultExistingMean["NextExisting14Value"] = pd.Series(GetNextNExistingValue(14))

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106


In [245]:
resultExistingMean["NextExisting21Value"] = pd.Series(GetNextNExistingValue(21))

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106


In [246]:
resultExistingMean["NextExisting30Value"] = pd.Series(GetNextNExistingValue(30))

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106


In [247]:
resultExistingMean.to_csv('data_transform/resultExistingMean.csv', index=None)

In [248]:
resultExistingMean["PrevExisting1Value"] = pd.Series(GetPrevNExistingValue(-1))

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106


In [ ]:
resultExistingMean["PrevExisting2Value"] = pd.Series(GetPrevNExistingValue(-2))

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106


In [ ]:
resultExistingMean["PrevExisting3Value"] = pd.Series(GetPrevNExistingValue(-3))

1 2

In [ ]:
resultExistingMean["PrevExisting7Value"] = pd.Series(GetPrevNExistingValue(-7))

In [ ]:
resultExistingMean["PrevExisting14Value"] = pd.Series(GetPrevNExistingValue(-14))

In [ ]:
resultExistingMean["PrevExisting21Value"] = pd.Series(GetPrevNExistingValue(-21))

In [ ]:
resultExistingMean["PrevExisting30Value"] = pd.Series(GetPrevNExistingValue(-30))

In [ ]:
resultExistingMean.to_csv('data_transform/resultExistingMean.csv', index=None)

## Next-Prev N Value

In [ ]:
def GetValueForNPrevNextDay(n, target, src):
    targetCopy = target.copy()
    targetCopy.Count.loc[targetCopy.IsValidation == 1] = np.nan

    targetCopy['Point'] = src.Point

    city = src.Point.unique()

    result = np.array([np.nan] * targetCopy.shape[0])
    for i in city:
        value = targetCopy.Count[targetCopy.Point == i].shift(n)
        for j in value.index:
            result[j] = value[j]


    prefics = "Prev" if n > 0 else "Next"
    targetCopy[prefics + str(abs(n)) + "Day"] = pd.Series(result)
    del targetCopy["Point"]
    return targetCopy[prefics + str(abs(n)) + "Day"]

In [ ]:
dst["Prev1Day"] = GetValueForNPrevNextDay(1, target, src)
dst["Prev2Day"] = GetValueForNPrevNextDay(2, target, src)
dst["Prev3Day"] = GetValueForNPrevNextDay(3, target, src)
dst["Prev7Day"] = GetValueForNPrevNextDay(7, target, src)

In [ ]:
dst["Next1Day"] = GetValueForNPrevNextDay(-1, target, src)
dst["Next2Day"] = GetValueForNPrevNextDay(-2, target, src)
dst["Next3Day"] = GetValueForNPrevNextDay(-3, target, src)
dst["Next7Day"] = GetValueForNPrevNextDay(-7, target, src)

In [ ]:
#target.to_csv('data_transform/target.csv', index = None)

In [ ]:
dst.head(20)

## Mean

In [ ]:
need_mean_list = src.columns.difference(['Date', 'Count', 'Point', 'ThroughDay'])
need_mean_list

### by date independed from points

In [ ]:
def calc_mean(src, group_column, target_column):
    mean = src.groupby([group_column])[target_column].mean()
    return src[group_column].apply(lambda x: mean[x])

In [ ]:

for column in need_mean_list:
    dst['MeanFor' + column] = calc_mean(src, column, 'Count')

In [ ]:
dst.head(15)

### by date depended from points

In [ ]:
def map_group_two_key(group, x, y):
    return group[x][y]

def calc_mean_by_columns(src, group_columns, target_column):
    mean = src.groupby(group_columns)[target_column].mean()
    return src[group_columns].apply(lambda x: map_group_two_key(mean, *x), axis=1)



In [ ]:
for column in need_mean_list:
    dst['MeanFor' + column + 'ByPoints'] = calc_mean_by_columns(src, [column, 'Point'], 'Count')

In [ ]:
dst['Point'] = pd.read_csv('data_original/common.csv')['Point']
dst.to_csv('data_transform/internal_features.csv', index=None)

In [ ]:
dst.head(15)

In [58]:
weather = pd.read_csv('data_transform/weather.csv')

In [59]:
weather

,Date,CityName,lat,lng,summary,icon,precipType,temperature,apparentTemperature,dewPoint,humidity,windSpeed,windBearing,visibility,cloudCover,pressure
0,2011-10-06,Сыктывкар,61.647851,50.833903,Overcast,cloudy,rain,4.95,1.77,4.13,0.94,4.05,195.0,8.32,1.00,990.66
1,2011-10-07,Сыктывкар,61.647851,50.833903,Overcast,cloudy,rain,7.22,5.09,5.04,0.86,3.13,240.0,11.27,1.00,NaN
2,2011-10-09,Сыктывкар,61.647851,50.833903,Partly Cloudy,partly-cloudy-night,rain,9.71,9.00,8.08,0.90,1.82,229.0,8.48,0.50,1007.93
3,2011-11-15,Сыктывкар,61.647851,50.833903,Overcast,cloudy,snow,-1.04,-5.66,-1.32,0.98,4.14,205.0,5.63,1.00,988.27
4,2011-12-08,Сыктывкар,61.647851,50.833903,Overcast,cloudy,snow,-1.78,-3.57,-2.19,0.97,1.38,171.0,10.54,1.00,1013.80
5,2011-12-09,Сыктывкар,61.647851,50.833903,Overcast,cloudy,snow,-2.44,-2.44,-4.04,0.89,0.89,188.0,9.06,1.00,1026.65
6,2011-12-10,Сыктывкар,61.647851,50.833903,Overcast,cloudy,snow,-4.43,-8.21,-6.47,0.86,2.45,163.0,10.61,1.00,1026.66
7,2011-12-11,Сыктывкар,61.647851,50.833903,Overcast,cloudy,snow,-7.28,-13.31,-8.93,0.88,4.10,173.0,8.42,1.00,1018.29
8,2011-12-13,Сыктывкар,61.647851,50.833903,Overcast,cloudy,snow,-9.77,-15.78,-11.34,0.88,3.49,192.0,8.40,1.00,1017.16
9,2011-12-14,Сыктывкар,61.647851,50.833903,Overcast,cloudy,snow,-7.00,-10.09,-7.81,0.94,1.69,195.0,10.77,1.00,1028.18


# All in one

In [89]:
data = pd.read_csv('data_transform/target.csv')
date = pd.read_csv('data_transform/date.csv')
city = pd.read_csv('data_transform/city.csv')
not_today = pd.read_csv('data_transform/not_today.csv')
internal_features = pd.read_csv('data_transform/internal_features.csv')
weather = pd.read_csv('data_transform/weather.csv')
weather = weather[['summary', 'windSpeed', 'pressure']]
weather = pd.get_dummies(weather)


need_columns = date.columns
data[need_columns] = date[need_columns]

data = pd.merge(data, city, on='Point', how='left')
data[['CountTomorrow', 'CountYesterday']] = not_today[['CountTomorrow', 'CountYesterday']]
need_columns = internal_features.columns
data[need_columns] = internal_features[need_columns]

#need_columns = weather.columns
#data[need_columns] = weather[need_columns]

#data = pd.merge(data, dollar, on='Date', how='left')

data.to_csv('data_transform/all.csv', index=None)

In [90]:
data.columns

Index([u'Count', u'Date', u'IsTest', u'IsTrain', u'IsValidation', u'Year',
       u'Month', u'MonthDay', u'WeekDay', u'Point', u'YearDay',
       u'ThroughMonth', u'ThroughWeek', u'ThroughDay', u'CityName', u'CityID',
       u'BranchNumber', u'CountTomorrow', u'CountYesterday', u'Prev1Day',
       u'Prev2Day', u'Prev3Day', u'Prev7Day', u'Next1Day', u'Next2Day',
       u'Next3Day', u'Next7Day', u'MeanForMonth', u'MeanForMonthDay',
       u'MeanForThroughMonth', u'MeanForThroughWeek', u'MeanForWeekDay',
       u'MeanForYear', u'MeanForYearDay', u'MeanForMonthByPoints',
       u'MeanForMonthDayByPoints', u'MeanForThroughMonthByPoints',
       u'MeanForThroughWeekByPoints', u'MeanForWeekDayByPoints',
       u'MeanForYearByPoints', u'MeanForYearDayByPoints'],
      dtype='object')

In [53]:
pd.isnull(data).sum() > 0

Count                           True
Date                           False
IsTest                         False
IsTrain                        False
IsValidation                   False
Year                           False
Month                          False
MonthDay                       False
WeekDay                        False
Point                          False
YearDay                        False
ThroughMonth                   False
ThroughWeek                    False
ThroughDay                     False
CityName_x                     False
CityID_x                       False
BranchNumber_x                 False
Prev1Day                        True
Prev2Day                        True
Prev3Day                        True
Prev7Day                        True
Next1Day                        True
Next2Day                        True
Next3Day                        True
Next7Day                        True
MeanForMonth                   False
MeanForMonthDay                False
M

In [45]:
data.head()

,Count,Date,IsTest,IsTrain,IsValidation,Year,Month,MonthDay,WeekDay,Point,...,MeanForWeekDay,MeanForYear,MeanForYearDay,MeanForMonthByPoints,MeanForMonthDayByPoints,MeanForThroughMonthByPoints,MeanForThroughWeekByPoints,MeanForWeekDayByPoints,MeanForYearByPoints,MeanForYearDayByPoints
0,8.0,2011-10-06,0,0,1,2011,10,6,3,Сыктывкар-1,...,92.560479,95.235294,95.668966,273.239583,235.609756,NaN,NaN,218.161111,95.235294,272.500000
1,88.0,2011-10-07,0,0,1,2011,10,7,4,Сыктывкар-1,...,122.657260,95.235294,90.311377,273.239583,245.613636,NaN,NaN,289.264865,95.235294,192.000000
2,1.0,2011-10-09,0,0,1,2011,10,9,6,Сыктывкар-1,...,124.063804,95.235294,155.605634,273.239583,243.128205,NaN,NaN,264.547619,95.235294,263.666667
3,1.0,2011-11-15,0,1,0,2011,11,15,1,Сыктывкар-1,...,87.724591,95.235294,132.172840,256.226190,235.021739,1.000,1.000000,194.982955,95.235294,192.000000
4,2.0,2011-12-08,0,1,0,2011,12,8,3,Сыктывкар-1,...,92.560479,95.235294,71.239437,238.595960,243.511628,101.125,18.333333,218.161111,95.235294,166.000000
